# Stay Subway Safe

## Interface
To interact with our wireframe (made with JustInMind), click here: https://www.justinmind.com/usernote/tests/47687661/47738106/47746104/index.html

## Algorithm and Explanation

These variable are defined by user input, as depicted in wireframe.

For this demonstration, values are hard coded and we assume the user wants to know what is happening NOW.

In [1]:
from datetime import datetime

In [2]:
station = "96 ST"
trainline = "1"
date = datetime.date(datetime.now())
time = datetime.time(datetime.now())

We then gather the data necessary to fulfil the user's request

#sort_by_week_day are not continuous, so call entries_exits_in_4h_chunks(filename,False)
STATION_CSV_DATA.get_final_station_data(url,1,"96 ST")
sort_by_weekday.sort_all_days("96 ST")
entries_exits_in_4h_chunks(filename,False)


In [3]:
import os
import sys
sys.path.insert(0, os.path.abspath('/Users/mehrkaur/Documents/projects/subway/MassTransit')) #ASK WENQING IF THERES A BETTER WAY

In [12]:
from get_station_csv import get_final_station_data
from sort_by_week_day import sort_all_days
from h4_chunks_in_py import entries_exits_in_4h_chunks

In [16]:
get_final_station_data(1, station, trainline)

In [6]:
#sort_all_days(station)